In [10]:
import os
import asyncio
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout

import time
import chardet

In [11]:
# defining range of recent NBA seasons to scrape (update the years to scape any number of season in recorded NBA history)

SEASONS = list(range(2024, 2025))

In [12]:
SEASONS

[2024]

In [13]:
# Creating directories on local drive for scraped data (STANDINGS_DIR was for initial training data to refine model. STANDINGS_UPDATED_DIR is for up to date data for future predictions of 2024 season)

DATA_DIR = "Data"
STANDINGS_DIR = os.path.join(DATA_DIR, "Standings")
STANDINGS_UPDATED_DIR = os.path.join(DATA_DIR, "Updated_Standings")
SCORES_DIR = os.path.join(DATA_DIR, "Scores")
SCORES_UPDATED_DIR = (DATA_DIR, "Updated_Scores")

os.makedirs(STANDINGS_DIR, exist_ok=True)
os.makedirs(STANDINGS_UPDATED_DIR, exist_ok=True)
os.makedirs(SCORES_DIR, exist_ok=True)

In [14]:
standings_files = os.listdir(STANDINGS_UPDATED_DIR)


In [15]:
# get HTML function: passing in URL, selector, sleep interval and a number of retries

async def get_html(url, selector, sleep=5, retries=6):
    html = None
    # initialise playwright, browser
    async with async_playwright() as p:
        browser = await p.firefox.launch(headless=True)
        page = await browser.new_page()
        for i in range(1, retries+1):
            
            # wrap it into a try except block
            try:
                await page.goto(url, timeout=30000)
                print(await page.title())
                html = await page.inner_html(selector, timeout=30000)
            except PlaywrightTimeout as e:
                print(f"Timeout error on {url}: {str(e)}")
                await asyncio.sleep(sleep * i)
            else:
                break
        await page.close()
        await browser.close()
    return html

In [16]:
season = 2024

url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"


In [17]:
# scrape season by season function (will then break this down into box_scores for each game per season)

async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all("a") 
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_UPDATED_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue 
            
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)
    

In [18]:
for season in SEASONS:
    await scrape_season(season)

Timeout error on https://www.basketball-reference.com/leagues/NBA_2024_games.html: Timeout 30000ms exceeded.
=========================== logs ===========================
navigating to "https://www.basketball-reference.com/leagues/NBA_2024_games.html", waiting until "load"
Timeout error on https://www.basketball-reference.com/leagues/NBA_2024_games.html: Timeout 30000ms exceeded.
=========================== logs ===========================
navigating to "https://www.basketball-reference.com/leagues/NBA_2024_games.html", waiting until "load"
2023-24 NBA Schedule | Basketball-Reference.com
2023-24 NBA Schedule | Basketball-Reference.com


In [19]:
standings_files = os.listdir(STANDINGS_UPDATED_DIR)

In [20]:
# parsing each individual box score per season 
async def scrape_game (standings_file):
    with open (standings_file, 'r', encoding='ISO-8859-1') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/") [-1])
        if os.path.exists(save_path):
            continue 

        html = await get_html(url, "#content")
        if not html:
            continue 
        with open(save_path, "w+") as f:
            f.write(html)
    

In [21]:
# downloading the box scores to drive

for f in standings_files:
    filepath = os.path.join(STANDINGS_UPDATED_DIR, f)
        
    await scrape_game(filepath)

Timeout error on https://www.basketball-reference.com/boxscores/202402110MIA.html: Timeout 30000ms exceeded.
=========================== logs ===========================
navigating to "https://www.basketball-reference.com/boxscores/202402110MIA.html", waiting until "load"
Celtics vs Heat, February 11, 2024 | Basketball-Reference.com
Kings vs Thunder, February 11, 2024 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202402120CHO.html: Timeout 30000ms exceeded.
=========================== logs ===========================
navigating to "https://www.basketball-reference.com/boxscores/202402120CHO.html", waiting until "load"
Pacers vs Hornets, February 12, 2024 | Basketball-Reference.com
76ers vs Cavaliers, February 12, 2024 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202402120ATL.html: Timeout 30000ms exceeded.
=========================== logs ===========================
navigating to "https://www.bas

['.DS_Store']